## XGBoost: eXtreme Gradient Boost 

Na dzisiejszych zajęciach będziemy pracować z klasyfikatorem XGBoost, który jest obecnie powszechnie używany.

**zad. 0** Zainstaluj pakiet `xgboost`.

**zad. 1** Zaimportuj pakiet `xgboost` jako `xgb`.

**zad. 2** Zaimportuj poniższe biblioteki.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

**zad. 3** Rozpakuj skompresowany plik `train.csv.gz`. Wczytaj za pomocą pandasa powstały plik do zmiennej `data`. Ponadto ustaw klucz na pierwszą kolumne.

**zad. 4** Przypisz do zmiennej `y` kolumnę `SalePrice`.

## One-hot Encoding

Część algorytmów jest w stanie obsłużyć tylko dane składające się z samych liczb. Stąd dane, które przyjmują wartości nie liczbowe należy przetworzyć dzięki tzw. *one-hot encoding*. Powiedzmy, że mamy czynnik (kolumnę), który przyjmuje tylko 5 wartości (np. kolorów). Kolory z natury nie da się uporządkość w żaden logiczny sposób i dlatego przypisanie wartości liczbowych byłoby błędem. Stąd, jak w naszym przykładzie, dla każdego koloru tworzymy dodatkową kolumnę i uzupełniamy jedynką tą kolumnę, która odpowiada danej wartości. W Pandasie służy do tego metoda `get_dummies`.

In [ ]:
dummies = data['Neighborhood'].str.get_dummies()

In [ ]:
dummies.head()

Dodajmy prefiks `Neighborhood_` do każdej kolumny z `dummies`.

In [ ]:
dummies.columns = ["Neighborhood_" + col for col in dummies.columns]

Dołączamy nowe kolumny do `data` i usuwamy kolumnę `Neighborhood`

In [ ]:
data = pd.concat([data, dummies], axis=1)
data = data.drop("Neighborhood", axis=1)

**zad. 5** Napisz funkcję `cat_to_columns(data, column)`, która zautomatyzuje powyższy proces. Argument data to `DataFrame`, zaś `column` to nazwa kolumny, którą chcemy zamienić.

In [ ]:
def cat_to_columns(data, column):
    return data

**zad. 6** Sprawdź dzianie na poniższych wartościach.

In [ ]:
data = cat_to_columns(data, "HouseStyle")
data = cat_to_columns(data, "BldgType")
data = cat_to_columns(data, "MSZoning")
data = cat_to_columns(data, "PavedDrive")

In [ ]:
columns = [
 'MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'Fireplaces',
 'GarageArea']

Stwórz zmienną `X` i przypisz do niej wszystkie kolumny z zmiennej `columns` i wszystkie nowo powstałe kolumny z poprzedniego zadania (plus te, które powstały z kolumny `Neighborhood`)

**zad. 7** Podaj ile mamy przykładów trenujących i liczbę zmiennych objaśniających (kolumn).

Dzielimy zbiór danych na zbiór treningowy i testowy:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

Tworzymy i trenujemy klasyfikator:

In [ ]:
xg_reg = xgb.XGBRegressor(objective="reg:linear", booster="gblinear", seed=123)

In [ ]:
xg_reg.fit(X_train, y_train)

Predykcja na zbiorze testowym:

In [ ]:
preds = xg_reg.predict(X_test)

**zad. 8** Oblicz `RMSE` dla danej predykcji.

**zad. 9** Dla porównania zbuduj model regresji liniowej i sprawdź, który model działa lepiej.

**zad. 10** zmień parametr `booster` na wartość `gbtree`.

Inny sposób trenowania modelu:

In [ ]:
import matplotlib
%matplotlib inline

housing_dmatrix = xgb.DMatrix(data=X, label=y)

params = {"objective":"reg:linear", "max_depth":2}

xg_reg = xgb.train(params=params, dtrain=housing_dmatrix, num_boost_round=10)

Wyświetlnie wytrenowanych drzew:

In [ ]:

# Plot the first tree
xgb.plot_tree(xg_reg, num_trees=0)

fig = matplotlib.pyplot.gcf()
fig.set_size_inches(15, 10)
plt.show()

# Plot the fifth tree
xgb.plot_tree(xg_reg, num_trees=4)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(15, 10)
plt.show()

# Plot the last tree sideways
xgb.plot_tree(xg_reg, num_trees=9, rankdir='LR')
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(15, 10)
plt.show()

Wykres wazności cech:

In [ ]:
# Create the DMatrix: housing_dmatrix
housing_dmatrix = xgb.DMatrix(data=X, label=y)

# Create the parameter dictionary: params
params = {"objective": "reg:linear", "max_depth": 4}

# Train the model: xg_reg
xg_reg = xgb.train(dtrain=housing_dmatrix, num_boost_round=10, params=params)

# Plot the feature importances
xgb.plot_importance(xg_reg)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(15, 10)
plt.show()


Walidacja krzyżowa:

In [ ]:
# Create the DMatrix: housing_dmatrix
housing_dmatrix = xgb.DMatrix(data=X, label=y)

# Create the parameter dictionary: params
params = {"objective":"reg:linear", "max_depth":4}

# Perform cross-validation: cv_results
cv_results = xgb.cv(dtrain=housing_dmatrix, params=params, nfold=4, num_boost_round=5, metrics="rmse", as_pandas=True, seed=123)

# Extract and print final boosting round metric
print(cv_results)

## Grid Search

*Grid search* jest podstawową techniką szukania hiperparametrów. Polega ona na sprawdzenieu wszystkich kombinacji wartości, które chcemy przetestować. W poniższym przykładzie chcemy znaleźć optymalne parametry dla trzech parametrów:  `colsample_bytree`, `n_estimators`, `max_depth`.

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create your housing DMatrix: housing_dmatrix
housing_dmatrix = xgb.DMatrix(data=X, label=y)

# Create the parameter grid: gbm_param_grid
gbm_param_grid = {
    'colsample_bytree': [0.3, 0.7],
    'n_estimators': [50],
    'max_depth': [2, 5]
}

# Instantiate the regressor: gbm
gbm = xgb.XGBRegressor()

# Perform grid search: grid_mse
grid_mse = GridSearchCV(param_grid=gbm_param_grid, estimator=gbm, scoring="neg_mean_squared_error", cv=4, verbose=1)


# Fit grid_mse to the data
grid_mse.fit(X, y)


# Print the best parameters and lowest RMSE
print("Best parameters found: ", grid_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_mse.best_score_)))

**Gratulacje!** Skończyłeś zadania na dzisiejsze zajęcia.